<a href="https://colab.research.google.com/github/giseletoledo/pythonSql/blob/main/catalogolivros.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Video de demonstração

https://drive.google.com/file/d/1kUbOL2f7ALFnn_E2ykxy7_yiLX5JSGut/view?usp=sharing


Utilizar a biblioteca sqlite

In [ ]:
import sqlite3




Criar o banco de dados



In [ ]:
def inicializar_banco():
    conn = sqlite3.connect('cadastro.db')
    cursor = conn.cursor()

    cursor.execute('''
        CREATE TABLE IF NOT EXISTS livros (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            titulo TEXT,
            autor TEXT,
            ano_publicacao TEXT,
            quantidade_total INTEGER,
            quantidade_disponivel INTEGER
        )
    ''')

    conn.commit()
    conn.close()

Cadastrar livros ou buscar livros



In [ ]:
def cadastrar_livro():
    titulo = input("Digite o título do livro: ")
    autor = input("Digite o autor do livro: ")
    ano_publicacao = input("Digite o ano de publicação do livro: ")
    quantidade_total = int(input("Digite a quantidade total de livros: "))
    quantidade_disponivel = quantidade_total  # Inicialmente, a quantidade disponível é igual à quantidade total

    conn = sqlite3.connect('cadastro.db')
    cursor = conn.cursor()

    cursor.execute('''
        INSERT INTO livros (titulo, autor, ano_publicacao, quantidade_total, quantidade_disponivel)
        VALUES (?, ?, ?, ?, ?)
    ''', (titulo, autor, ano_publicacao, quantidade_total, quantidade_disponivel))

    conn.commit()
    conn.close()

    print("Livro cadastrado com sucesso!")

Buscar livros

In [ ]:
def buscar_livro():
    termo = input("Digite um termo para buscar livros: ")

    conn = sqlite3.connect('cadastro.db')
    cursor = conn.cursor()

    cursor.execute('''
        SELECT * FROM livros
        WHERE LOWER(titulo) LIKE ? OR LOWER(autor) LIKE ?
    ''', ('%' + termo.lower() + '%', '%' + termo.lower() + '%'))

    resultados = cursor.fetchall()

    conn.close()

    if resultados:
        for livro in resultados:
            print(livro)
    else:
        print("Nenhum livro encontrado com o termo inserido.")

Verificar a disponibilidade

In [ ]:
def verificar_disponibilidade(titulo):
    conn = sqlite3.connect('cadastro.db')
    cursor = conn.cursor()

    cursor.execute('''
        SELECT quantidade_total, quantidade_disponivel FROM livros
        WHERE LOWER(titulo) = ?
    ''', (titulo.lower(),))

    resultado = cursor.fetchone()

    conn.close()

    if resultado:
        return resultado
    else:
        return (0, 0)

Emprestar livro

In [ ]:
def emprestar_livro(titulo, quantidade):
    conn = sqlite3.connect('cadastro.db')
    cursor = conn.cursor()

    # Verificar a disponibilidade antes de emprestar
    quantidade_total, quantidade_disponivel = verificar_disponibilidade(titulo)

    if quantidade_disponivel >= quantidade:
        # Atualizar a quantidade disponível após o empréstimo
        nova_quantidade_disponivel = quantidade_disponivel - quantidade
        cursor.execute('''
            UPDATE livros
            SET quantidade_disponivel = ?
            WHERE LOWER(titulo) = ?
        ''', (nova_quantidade_disponivel, titulo.lower()))

        print(f"{quantidade} livro(s) de '{titulo}' emprestado(s) com sucesso!")
    else:
        print("Quantidade insuficiente de livros disponíveis.")

    conn.commit()
    conn.close()

Menu de opções para cadastrar ou buscar um livro


In [ ]:
inicializar_banco()

opcao = input("Digite 'c' para cadastrar um livro, 'b' para buscar um livro, 'v' para verificar a disponibilidade ou 'e' para emprestar um livro: ")

if opcao.lower() == 'c':
    cadastrar_livro()
elif opcao.lower() == 'b':
    buscar_livro()
elif opcao.lower() == 'v':
    titulo = input("Digite o título do livro para verificar a disponibilidade: ")
    quantidade_total, quantidade_disponivel = verificar_disponibilidade(titulo)
    print(f"A quantidade total de '{titulo}' é: {quantidade_total}")
    print(f"A quantidade disponível de '{titulo}' é: {quantidade_disponivel}")
elif opcao.lower() == 'e':
    titulo = input("Digite o título do livro para emprestar: ")
    quantidade = int(input("Digite a quantidade de livros para emprestar: "))
    emprestar_livro(titulo, quantidade)
else:
    print("Opção inválida.")

Digite 'c' para cadastrar um livro, 'b' para buscar um livro, 'v' para verificar a disponibilidade ou 'e' para emprestar um livro: v
Digite o título do livro para verificar a disponibilidade: Senhor dos Anéis
A quantidade total de 'Senhor dos Anéis' é: 4
A quantidade disponível de 'Senhor dos Anéis' é: 2


Limpar o banco de dados

In [ ]:
def limpar_banco():
    conn = sqlite3.connect('cadastro.db')
    cursor = conn.cursor()

    cursor.execute('DROP TABLE IF EXISTS livros')

    conn.commit()
    conn.close()

limpar_banco()
